In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import Ridge

# 1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv.

In [2]:
data_train = pd.read_csv('salary-train.csv')

In [3]:
data_train

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355
...,...,...,...,...
59995,"As a result of continued growth, First Class S...",Whitley Bay,contract,26400
59996,PHP / MVC Web Developer MacclesfieldCirca ***...,Macclesfield,permanent,26000
59997,"Staff Nurse, Nursing Home, Baldock White Recru...",Baldock,NaN,24500
59998,This is one of the best agency side opportunit...,The City,permanent,65000


# 2. Проведите предобработку

Приведите тексты к нижнему регистру (text.lower()).

Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:

In [107]:
data_train['FullDescription'].str.lower()
data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

0        international sales manager london     k      ...
1        an ideal opportunity for an individual that ha...
2        online content and brand manager   luxury reta...
3        a great local marketleader is seeking a perman...
4        registered nurse   rgn  nursing home for young...
                               ...                        
59995    as a result of continued growth  first class s...
59996    php   mvc web developer  macclesfieldcirca    ...
59997    staff nurse  nursing home  baldock white recru...
59998    this is one of the best agency side opportunit...
59999    must have cscs card must have asbestos awarene...
Name: FullDescription, Length: 60000, dtype: object

Примените TfidfVectorizer для преобразования текстов в векторы признаков. 
Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [110]:
vectorizer = TfidfVectorizer(min_df=5)
dtrain_vectorizer = vectorizer.fit_transform(data_train["FullDescription"])

 Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.

In [49]:
for i in ['LocationNormalized','ContractTime']:
    data_train[i].fillna('nan', inplace=True)

Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.


In [111]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [112]:
X = hstack([dt_vectorizer, X_train_categ])

# 3. Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [113]:
clf = Ridge(alpha=1, random_state=241)
clf.fit(X, data_train['SalaryNormalized'])

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=241, solver='auto', tol=0.001)

# 4.  Постройте прогнозы для двух примеров из файла salary-test-mini.csv.
Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [114]:
X_test_row = pd.read_csv('salary-test-mini.csv')

In [128]:
X_test_row['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
X_test_row['FullDescription'].str.lower()
dtest_vectorizer = vectorizer.transform(X_test_row["FullDescription"])
X_test_categ = enc.transform(X_test_row[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([dtest_vectorizer, X_test_categ])

In [129]:
y_test = clf.predict(X_test)

In [130]:
f = open('Answer.txt', 'w')
f.write(str(np.round(y_test[0], 2)) +' ' + str(np.round(y_test[1], 2)))
f.close()